<a href="https://colab.research.google.com/github/gulmert89/dsBootcamp/blob/master/project5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Graduation Project: Image Processing with Convolutional Neural Networks

Images of happy, shocked & sad faces.

---
# 1. Introduction

## 1.1 Imported Goods

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
%cd "/content/drive/My Drive/MertColab/project5/"

/content/drive/My Drive/MertColab/project5


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime, os
from keras.utils import to_categorical
from keras import models, layers